In [ ]:
import numpy as np
import pandas as pd
import random
import re
import math
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from itertools import combinations

In [ ]:
pip install --upgrade xlrd

In [ ]:
df = pd.read_excel('data_excel.xlsx')

In [ ]:
#function to get column
def column(matrix, i):
    return [row[i] for row in matrix]

#jaccard similarity function
def jaccard(x, y):
  x_set = set(x)
  y_set = set(y)
  similarity = len(x_set.intersection(y_set)) / len(x_set.union(y_set))
  return similarity

In [ ]:
titles = (df['title'].str.lower())

titles = titles.str.replace('inches', 'inch')
titles = titles.str.replace('"', 'inch')
titles = titles.str.replace('-inch', 'inch')
titles = titles.str.replace(' inch', 'inch')

titles = titles.str.replace('Hertz', 'hz')
titles = titles.str.replace('hertz', 'hz')
titles = titles.str.replace('Hz', 'hz')
titles = titles.str.replace('HZ', 'hz')
titles = titles.str.replace(' hz', 'hz')
titles = titles.str.replace('-hz', 'hz')
titles = titles.str.replace('-', '')

titles_list = list(titles)

titles_word_list = [word for line in titles_list for word in line.split()]
model_words = list()

for word in titles_word_list:
    z = re.match("[a-zA-Z0-9]*(([0-9]+[ˆ0-9, ]+)|([ˆ0-9, ]+[0-9]+))[a-zA-Z0-9]*", word)
    if z:
        model_words.append(word)

#remove duplicates
model_words_set = set(model_words)
#convert set back to a list
model_words = list(model_words_set)

In [ ]:
def clean_titles(bootstrap):
  titles = (bootstrap['title'].str.lower())

  titles = titles.str.replace('inches', 'inch')
  titles = titles.str.replace('"', 'inch')
  titles = titles.str.replace('-inch', 'inch')
  titles = titles.str.replace(' inch', 'inch')

  titles = titles.str.replace('Hertz', 'hz')
  titles = titles.str.replace('hertz', 'hz')
  titles = titles.str.replace('Hz', 'hz')
  titles = titles.str.replace('HZ', 'hz')
  titles = titles.str.replace(' hz', 'hz')
  titles = titles.str.replace('-hz', 'hz')
  titles = titles.str.replace('-', '')

  return list(titles)

In [ ]:
def df_to_matrix(df):
  return df.values

In [ ]:
def binary_matrix(words: list, observations: list):
    matrix = np.zeros((len(words), len(observations)))
    for i in range(len(words)):
        for j in range(len(observations)):
            if words[i] in observations[j]:
                matrix[i][j] = 1
    return matrix

In [ ]:
def rand_int_list(rows_signature: int):
    integer_list = list()
    for i in range(rows_signature):
        random_int = random.randint(1, rows_signature)

        while random_int in integer_list:
            random_int = random.randint(1, rows_signature)

        integer_list.append(random_int)
    return integer_list

In [ ]:
def signature_matrix(matrix, a: list, b: list, k: int, prime: int):
    num_rows, num_cols = matrix.shape
    signature = np.full((k, num_cols), 613)

    for i in range(num_rows):
        for j in range(num_cols):
            if matrix[i][j] == 1:
              for p in range(k):
                    if ((a[p] + b[p] * i ) % prime) < signature[p][j]:
                        signature[p][j] = ((a[p] + b[p] * i ) % prime)

    return signature

In [ ]:
def split_signature(signature, r):
  length = len(signature)
  b = math.floor(length/r)
  split_sig = []
  for i in range(0, length, r):
    split_sig.append(signature[i : i+r])
  return split_sig


In [ ]:
def LSH(matrix, r, n):
  
  b = math.floor(600/r)

  buckets = []
  column = 0 

  for l in range(b):
    buckets.append({})

  for i in range(b):
    column = 0
    for j in range(n):
      string = ""
      for q in range(r):
        string = string + str(matrix[i][q][j])
      
      if string not in buckets[i].keys():
        buckets[i][string] = []
      buckets[i][string].append(column)
  
      column = column + 1
  
  return buckets

In [ ]:
def candidate_pairs(buckets, r, n):
  candidate_matrix = np.zeros((n, n))
  b = math.floor(600/r)
  for i in range(b):
    for key in buckets[i]:
      if len(buckets[i].get(key)) > 1:
        for j in range(len(buckets[i].get(key))-1):
          for p in range(j+1, len(buckets[i].get(key))):
            candidate_matrix[buckets[i].get(key)[j]][buckets[i].get(key)[p]] = 1
                    

  
  return candidate_matrix

In [ ]:
def performance(can_matrix, matrix, n):
  pot_pairs = 0
  total_pairs = 0
  true_pos = 0

  for i in range(n):
    for j in range(n):
      if can_matrix[i][j] == 1:
        pot_pairs = pot_pairs + 1
     
        if matrix[i][0] == matrix[j][0]:
          true_pos = true_pos + 1
      
  for i in range(n):
    for j in range(i+1, n):
      if matrix[i][0] == matrix[j][0]:
        total_pairs = total_pairs + 1

  pq = true_pos / pot_pairs
  pc = true_pos/ total_pairs

  F1_star = (2 * pq * pc) / (pq + pc)

  fraq = pot_pairs / (n * (n-1) / 2)

  return[true_pos, total_pairs, pot_pairs, pq, pc, F1_star, fraq ]





In [ ]:
def dissimilarity_matrix(can_matrix, matrix, signature, n):
    
  dis_matrix = np.full((n, n), 1000)

  for i in range(n):
    for j in range(n):
      if can_matrix[i][j] == 1:

        #check if candidates are from the same shop
        if matrix[i][1] == matrix[j][1]:
          dis_matrix[i][j] = 1000
          dis_matrix[j][i] = 1000
        #check if candidates are from the same brand
        elif (matrix[i][54] == matrix[i][54] and matrix[j][54] == matrix[j][54]) and (matrix[i][54] != matrix[j][54]):
          dis_matrix[i][j] = 1000
          dis_matrix[j][i] = 1000
        elif (matrix[i][26] == matrix[i][26] and matrix[j][26] == matrix[j][26]) and (matrix[i][26] != matrix[j][26]):
          dis_matrix[i][j] = 1000
          dis_matrix[j][i] = 1000
        else:
          #certain similiarity measure 
          dis_matrix[i][j] = 1 - jaccard(column(signature, i), column(signature, j))
          dis_matrix[j][i] = 1 - jaccard(column(signature, i), column(signature, j))


  return dis_matrix

In [ ]:
def cluster_algorithm(threshold, dissimilarity_matirx):
  cluster = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='average', distance_threshold=threshold, compute_full_tree = True)
  clusters = cluster.fit(dissimilarity_matirx)

  pairs = []

  for i in range(cluster.n_clusters_):
  
    no_products_cluster = len((np.where(cluster.labels_==i)[0]))
    products_cluster = np.where(cluster.labels_==i)

    if no_products_cluster > 1:
      pairs = pairs + list(combinations(products_cluster[0], 2))

  return pairs


In [ ]:
def f1_performance(pairs, matrix, n):
  true_pos = 0
  false_pos = 0
  false_neg = 0
  total_pairs = 0

  pot_pairs = len(pairs)

  for i in range(len(pairs)):
    if matrix[pairs[i][0]][0] ==  matrix[pairs[i][1]][0]:
      true_pos = true_pos + 1
    else:
      false_pos = false_pos + 1

  
  for i in range(n):
    for j in range(i+1, n):
      if matrix[i][0] == matrix[j][0]:
        total_pairs = total_pairs + 1
  
  false_neg = total_pairs - true_pos
  


  F1 = (2 * true_pos) / (2 * true_pos + false_pos + false_neg)

  fraq = pot_pairs / (n * (n-1) / 2)

  return F1, fraq 

In [ ]:
def graph_results(df):

  #get bootstrap

  train = df.sample(n=1624, replace = True)
  train = train.drop_duplicates()
  train = train.sort_index(ascending=True)

  test = pd.concat([df, train, train]).drop_duplicates(keep=False)

  #clean titles

  train_titles = clean_titles(train) 

  test_titles = clean_titles(test)

  #convert dataframe to matrices 

  train_matrix = df_to_matrix(train)
  
  test_matrix = df_to_matrix(test)

  #get sizes of train and test matrices

  n_train, c_train = train_matrix.shape

  n_test, c_test = test_matrix.shape

  #create binary vectors 

  binary_train = binary_matrix(model_words, train_titles)

  binary_test = binary_matrix(model_words, test_titles)

  #create list of random int for minhash, k = 600 (signature matrix size)

  k = 600

  a_list = rand_int_list(k)
  b_list = rand_int_list(k)

  #create signature matrix, take large prime number

  prime = 1283

  signature_train = signature_matrix(binary_train, a_list, b_list, k, prime)

  signature_test = signature_matrix(binary_test, a_list, b_list, k, prime)

  
  
  
  #loop over different values of r te get results for graphs over the test set

  graph_results = []

  for r in range(1, k, 1):

    r_list = []

    r_list.append(r)


    #split signature matrix

    split_test = split_signature(signature_test, r)

    #do LSH

    buckets_test = LSH(split_test, r, n_test)

    #create candidate matrix

    candidate_matrix_test = candidate_pairs(buckets_test, r, n_test)

    r_list.append(performance(candidate_matrix_test, test_matrix, n_test))

    graph_results.append(r_list)

    print(r)

  return ["graph results", [graph_results]]






  

In [ ]:
graph1 = graph_results(df)
graph2 = graph_results(df)
graph3 = graph_results(df)
graph4 = graph_results(df)
graph5 = graph_results(df)


bootstrap result

In [ ]:
def bootstrap_results(df):

  #get bootstrap

  train = df.sample(n=1624, replace = True)
  train = train.drop_duplicates()
  train = train.sort_index(ascending=True)

  test = pd.concat([df, train, train]).drop_duplicates(keep=False)

  #clean titles

  train_titles = clean_titles(train) 

  test_titles = clean_titles(test)

  #convert dataframe to matrices 

  train_matrix = df_to_matrix(train)
  
  test_matrix = df_to_matrix(test)

  #get sizes of train and test matrices

  n_train, c_train = train_matrix.shape

  n_test, c_test = test_matrix.shape

  #create binary vectors 

  binary_train = binary_matrix(model_words, train_titles)

  binary_test = binary_matrix(model_words, test_titles)

  #create list of random int for minhash, k = 600 (signature matrix size)

  k = 600

  a_list = rand_int_list(k)
  b_list = rand_int_list(k)

  #create signature matrix, take large prime number

  prime = 1283

  signature_train = signature_matrix(binary_train, a_list, b_list, k, prime)

  signature_test = signature_matrix(binary_test, a_list, b_list, k, prime)

  #print("preliminary done")


  #loop over different values of r to get f1 results over the training sample

  training_results = []

  for r in range(1, k, 1):

    r_list = []

    r_list.append(r)


    #split signature matrix

    split_training = split_signature(signature_train, r)

    #print("training set is split")

    #do LSH

    buckets_training = LSH(split_training, r, n_train)

    #print("buckets are made")

    #create candidate matrix

    candidate_matrix_training = candidate_pairs(buckets_training, r, n_train)

    #print("candidate matrix is done")

    #create dissimilarity matrix 

    dis_matrix = dissimilarity_matrix(candidate_matrix_training, train_matrix, signature_train, n_train)

    #print("dissimilarity matrix is made")

    #loop over values of the threshold

    print("now optimize for")  
    print(r)


    for j in np.arange(50, 1000, 50):

      print(r)
      print(j)

      j_list = []

      j_list.append(j)

      pairs = cluster_algorithm(j, dis_matrix)

      F1 = f1_performance(pairs, train_matrix, n_train)

      j_list.append(F1)

      r_list.append(j_list)
    

    training_results.append(r_list)

  return training_results



In [ ]:
bootstrap1 = bootstrap_results(df)
bootstrap2 = bootstrap_results(df)
bootstrap3 = bootstrap_results(df)
bootstrap4 = bootstrap_results(df)
bootstrap5 = bootstrap_results(df)


function to find best thresholds

In [ ]:
def average_thresholds(list1, list2, list3, list4, list5):
  average_results = []

  for i in range(len(list1)):

    results = []


    for j in range(1, 20, 1):     

      #results.append(list1[i][0])

      total_f1 = list1[i][j][1][0] + list2[i][j][1][0] + list3[i][j][1][0] + list4[i][j][1][0] + list5[i][j][1][0]
      average_f1 = total_f1 / 5

      results.append(list1[i][j][0])
      results.append(average_f1)

    average_results.append(results)
  
  return average_results


In [ ]:
def selecting_thresholds(antwoorden):
  best_thresholds = []
  for i in range(len(antwoorden)):
    best_threshold = antwoorden[i][1]
    index = 1

    for j in range(3, 38, 2):

      if antwoorden[i][j] > best_threshold:
        best_threshold = antwoorden[i][j]
        index = j
     
      
    
    best_thresholds.append(antwoorden[i][index - 1])
  return best_thresholds



In [ ]:
average_results = average_thresholds(bootstrap1, bootstrap2, bootstrap3, bootstrap4, bootstrap5)
best_thresholds = selecting_thresholds(average_results)


In [ ]:
def test_results(df, optimal_thresholds):
  
  #get bootstrap

  train = df.sample(n=1624, replace = True)
  train = train.drop_duplicates()
  train = train.sort_index(ascending=True)

  test = pd.concat([df, train, train]).drop_duplicates(keep=False)

  #clean titles

  train_titles = clean_titles(train) 

  test_titles = clean_titles(test)

  #convert dataframe to matrices 

  train_matrix = df_to_matrix(train)
  
  test_matrix = df_to_matrix(test)

  #get sizes of train and test matrices

  n_train, c_train = train_matrix.shape

  n_test, c_test = test_matrix.shape

  #create binary vectors 

  binary_train = binary_matrix(model_words, train_titles)

  binary_test = binary_matrix(model_words, test_titles)

  #create list of random int for minhash, k = 600 (signature matrix size)

  k = 600

  a_list = rand_int_list(k)
  b_list = rand_int_list(k)

  #create signature matrix, take large prime number

  prime = 1283

  signature_train = signature_matrix(binary_train, a_list, b_list, k, prime)

  signature_test = signature_matrix(binary_test, a_list, b_list, k, prime)

  print("preliminary done")


  #loop over different values of r to get f1 results over the training sample

  test_results = []

  i = 0

  for r in range(1, k, 1):

    r_list = []

    r_list.append(r)


    #split signature matrix

    split_test = split_signature(signature_test, r)

    print("test set is split")

    #do LSH

    buckets_test = LSH(split_test, r, n_test)

    print("buckets are made")

    #create candidate matrix

    candidate_matrix_test = candidate_pairs(buckets_test, r, n_test)

    print("candidate matrix is done")

    #create dissimilarity matrix 

    dis_matrix = dissimilarity_matrix(candidate_matrix_test, test_matrix, signature_test, n_test)

    print("dissimilarity matrix is made")

    #get optimal threshold

    j = optimal_thresholds[i]

    pairs = cluster_algorithm(j, dis_matrix)

    print(pairs)

    F1 = f1_performance(pairs, test_matrix, n_test)

    r_list.append(F1)

    test_results.append(r_list)

    i = i + 1

  return test_results

  

    




In [ ]:
final1 = test_results(df, best_thresholds)
final2 = test_results(df, best_thresholds)
final3 = test_results(df, best_thresholds)
final4 = test_results(df, best_thresholds)
final5 = test_results(df, best_thresholds)

GRAPHING

In [ ]:
def average_f1(final1, final2, final3, final4, final5):
    average = []
    for i in range(1, 38, 1):
      sum = 0
      sum = sum + final1[i][1][0] + final2[i][1][0]  + final3[i][1][0]  + final4[i][1][0]  + final5[i][1][0] 
      av = sum / 5
      average.append(av)

    return average


In [ ]:
def average_completeness(graph1, graph2, graph3, graph4, graph5):
  average = []
  for i in range(599):
    sum = 0
    sum = sum + graph1[1][0][i][1][4] + graph2[1][0][i][1][4]  + graph3[1][0][i][1][4]  + graph4[1][0][i][1][4]  + graph5[1][0][i][1][4] 
    av = sum / 5
    average.append(av)

  return average
  

In [ ]:
def average_f1star(graph1, graph2, graph3, graph4, graph5):
  average = []
  for i in range(599):
    sum = 0
    sum = sum + graph1[1][0][i][1][5] + graph2[1][0][i][1][5]  + graph3[1][0][i][1][5]  + graph4[1][0][i][1][5]  + graph5[1][0][i][1][5] 
    av = sum / 5
    average.append(av)

  return average
  

In [ ]:
def average_quality(graph1, graph2, graph3, graph4, graph5):
  average = []
  for i in range(599):
    sum = 0
    sum = sum + graph1[1][0][i][1][3] + graph2[1][0][i][1][3]  + graph3[1][0][i][1][3]  + graph4[1][0][i][1][3]  + graph5[1][0][i][1][3] 
    av = sum / 5
    average.append(av)

  return average

In [ ]:
pair_completeness = average_completeness(graph1, graph2, graph3, graph4, graph5)

for i in range(599):
  x.append(graph1[1][0][i][1][6])
 
plt.plot(x, pair_completeness, color="black")


# naming the x axis
plt.xlabel('fraction of comparisons')
# naming the y axis
plt.ylabel('pair completeness')
plt.legend()
# giving a title to my graph
plt.title('')


# function to show the plot
plt.show()

In [ ]:
f1star = average_f1star(graph1, graph2, graph3, graph4, graph5)

for i in range(2, 20, 1):
  x.append(graph1[1][0][i][1][6])

plt.plot(x, f1star, color="black")


# naming the x axis
plt.xlabel('fraction of comparisons')
# naming the y axis
plt.ylabel('f1*')
plt.legend()
# giving a title to my graph
plt.title('')


# function to show the plot
plt.show()

In [ ]:
pair_quality = average_quality(graph1, graph2, graph3, graph4, graph5)

for i in range(599):
  x.append(graph1[1][0][i][1][6])
 
plt.plot(x, pair_quality, color="black", label="pair quality")


# naming the x axis
plt.xlabel('fraction of comparisons')
# naming the y axis
plt.ylabel('pair quality')
plt.legend()
# giving a title to my graph
plt.title('')


# function to show the plot
plt.show()

In [ ]:
f1 = average_f1(bootstrap1, bootstrap2, bootstrap3, bootstrap4, bootstrap5)

for i in range(1, 37, 1):
  x.append(graph1[1][0][i][1][6])
  
 
print(x[20])
plt.plot(x, f1, color="black")


# naming the x axis
plt.xlabel('fraction of comparisons')
# naming the y axis
plt.ylabel('f1')
plt.legend()
# giving a title to my graph
plt.title('')


# function to show the plot
plt.show()